# Physical Climate Risk Propagation Model - EU-27 SAM (Scenario Runs)
## End-to-end simulation, post-processing, and visualization

This notebook provides a **clean, reproducible workflow** to:
1. Load and prepare the Eurostat-based Social Accounting Matrix (SAM)
2. Initialize the IO Climate Risk Propagation Model
3. Define and run climate-related shock scenarios
4. Post-process model outputs into economic impact indicators
5. Visualize results in a dashboard-ready format

All heavy logic is delegated to the `src/io_climate` package.
This notebook acts as an **orchestrator and presentation layer**, not a model implementation.

This notebook is the *single entry point* to:
- load the latest EU-27 SAM (long format) from Databricks
- extract IO-style blocks (`Z`, `FD`, `X`, `A`, `globsec_of`, `node_labels`)
- run the **physical-risk propagation** model under supply/demand shocks
- inspect and visualize impacts (country / sector), including a Europe choropleth



## 0) Environment and imports

**Prerequisites**
- You are running on a Databricks cluster or a local environment with a working Spark session (`spark`) connected to the Databricks metastore.
- The project `src/` folder is importable (we add it to `sys.path` below).

If you are running locally, ensure Databricks Connect is configured and `spark` is available.


In [1]:
import os
import sys

import numpy as np
import pandas as pd
from pathlib import Path


# --- Make project imports work from the repo root
PROJECT_ROOT = os.path.abspath(os.getcwd())
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# --- Locate inputs (repo-friendly) -------------------------------------------
CANDIDATES = [Path("src/data_io"), Path("data_io"), Path(".")]
for base in CANDIDATES:
    if (base / "input_data.xlsx").exists() and (base / "Output_API.txt").exists():
        DATA_IO_DIR = base
        break
else:
    raise FileNotFoundError(
        "Could not find input_data.xlsx and Output_API.txt. "
        "Expected in src/data_io/ or data_io/."
    )

LOSS_XLSX = DATA_IO_DIR / "input_data.xlsx"
API_TXT   = DATA_IO_DIR / "Output_API.txt"

print("Using DATA_IO_DIR:", DATA_IO_DIR.resolve())


from data_io.eurostat_sam import load_sam_latest_year, extract_model_inputs_from_sam
from src.data_io.sector_decoder import build_sector_decoder
from io_climate.postprocess import postprocess_results
from src.io_climate.viz import build_dashboard_bundle
from src.data_io.eurostat_output import EurostatOutputConfig, fetch_output_2024_prices_mn_eur
from src.io_climate.calibration import (
    CalibrationPaths,
    load_losses_from_excel,
    build_intensity_panel,
    build_percentile_table,
    shock_scalar,
)
from io_climate.model import IOClimateModel


Using DATA_IO_DIR: C:\Users\GiorgioCesareCarlucc\OneDrive - OpenEconomics S.r.l\GitHub\Physical-Climate-Risk\src\data_io


## 1) Load SAM and extract model inputs

The SAM is expected in **long format** and must include:
- `geo_ava`, `ind_ava` (producer country/account)
- `geo_use`, `ind_use` (user country/account)
- `value` (flow)
- `share` (precomputed coefficient: cell value / total output of user column)

`extract_model_inputs_from_sam(...)` builds:
- `Z` (n×n) intermediate flows among production sectors (`P_*`)
- `FD` (n,) final demand from accounts `[HH, GOV, CF, WRL_REST]`
- `X` (n,) gross output = `row_sum(Z) + FD`
- `A` (n×n) technical coefficients (from `share` for the `P_* → P_*` block)
- `globsec_of` (n,) maps each node to its global sector id (same `P_*` across countries)
- `node_labels` list of `"CC::P_..."` strings, consistent with matrix ordering


In [2]:
# Load latest year from Databricks table
sam_df, latest_year = load_sam_latest_year(spark)
print("SAM year:", latest_year)
print("Rows:", sam_df.count())

# Extract model inputs (numpy arrays + labels)
Z, FD, X, A, globsec_of, node_labels = extract_model_inputs_from_sam(sam_df)

n = len(node_labels)
print("n nodes:", n)
print("Z shape:", Z.shape, "A shape:", A.shape, "FD shape:", FD.shape, "X shape:", X.shape)


SAM year: 2022
Rows: 2040159
n nodes: 1695
Z shape: (1695, 1695) A shape: (1695, 1695) FD shape: (1695,) X shape: (1695,)


**Sector name mapping table**

If available, you can load a sector mapping/decoder table to attach human-readable sector names to results. 
At this stage, the model uses **NACE codes** from `node_labels` (e.g. `IT::P_D35`). The mapping is **not** required to run the model.


In [3]:
sectors_in_sam = sorted({lbl.split("::")[1] for lbl in node_labels})
sector_decoder = build_sector_decoder(
    spark,
    sectors_in_sam=sectors_in_sam,
)
print("Sectors in SAM:", len(sectors_in_sam))

Sectors in SAM: 63


**Quick sanity checks**

These checks catch the most common ingestion issues (ordering mismatches, missing blocks, negative values).


In [4]:
assert Z.shape == (n, n)
assert A.shape == (n, n)
assert FD.shape == (n,)
assert X.shape == (n,)
assert len(globsec_of) == n

print("Min values — Z:", float(Z.min()), "A:", float(A.min()), "FD:", float(FD.min()), "X:", float(X.min()))
print("Totals — sum(Z):", float(Z.sum()), "sum(FD):", float(FD.sum()), "sum(X):", float(X.sum()))

# Accounting identity at baseline (row accounting)
row_gap = np.linalg.norm(X - (FD + Z.sum(axis=1)), ord=1)
print("Baseline accounting gap (L1):", float(row_gap))


Min values — Z: 0.0 A: 0.0 FD: -0.040412221578703625 X: 1.2000000000907844
Totals — sum(Z): 13941501.474966172 sum(FD): 14831340.773248244 sum(X): 28772842.248214405
Baseline accounting gap (L1): 0.0


## 2) Instantiate the model

`IOClimateModel` stores the **baseline economy** internally (`Z0`, `A0`, `L0`, `X0`, `FD0`), and enforces a fixed **global-technology** matrix `A_G` derived from baseline global-sector aggregation.

For scenario diagnostics, keep a copy of baseline output **outside** the model, to avoid any ambiguity.


In [5]:
model = IOClimateModel(
    Z=Z,
    FD=FD,
    X=X,
    globsec_of=globsec_of,
    A=A,
    node_labels=node_labels,
)

# Baseline copies for diagnostics
X_baseline = X.copy()
FD_baseline = FD.copy()

print("Model ready.")
print("Baseline total output:", float(X_baseline.sum()))
print("Baseline total final demand:", float(FD_baseline.sum()))
print("Global sectors:", model.S_glob)


Model ready.
Baseline total output: 28772842.248214405
Baseline total final demand: 14831340.773248244
Global sectors: 63


## 3) Define and run a scenario

You can define shocks either:
- by providing `sd` and `sp` vectors directly, or
- by specifying `(country_codes, sector_codes, supply_shock_pct, demand_shock_pct)`.

This notebook uses **scenario mode** for convenience.

Notes:
- Supply shock reduces capacity: `X_cap = X0 * (1 - sp)`
- Demand shock reduces initial final demand: `FD_post = FD0 * (1 - sd)`
- Outer iteration reduces demand *further* only where implied demand is lower than `FD_post` (elementwise monotone update).


## 3B) Hazard-calibrated supply shock (losses + Eurostat output)

This optional path builds **hazard-calibrated supply shocks** from historical loss data and national accounts output:

**Inputs**
- `src/data_io/input_data.xlsx`: economic losses (million EUR, 2024 prices)
- `src/data_io/Output_API.txt`: Eurostat SDMX API URL for `nama_10_a64` (P1 output)

**Method (v1)**
1. Compute **hazard shares** per country from cumulative losses (1980–2024).
2. Allocate each country’s annual total losses across hazards using those shares.
3. Normalize hazard-specific annual losses by **country total output** (constant 2024 prices).
4. Compute percentile-based intensity levels (moderate / severe / extreme / very_extreme).
5. Apply a **flat supply shock across all sectors** of the selected country (no exogenous demand shock).


In [6]:
# --- 1) Load losses ---------------------------------------------------------
paths = CalibrationPaths(losses_xlsx_path=str(LOSS_XLSX))
df_type, df_year = load_losses_from_excel(paths)

# --- 2) Load Eurostat output and convert to constant 2024 prices --------------
api_url = API_TXT.read_text().strip()

cache_dir = Path("data/cache")
cache_dir.mkdir(parents=True, exist_ok=True)
cache_path = cache_dir / "eurostat_p1_output_2024.parquet"

out_cfg = EurostatOutputConfig(api_url=api_url, cache_path=str(cache_path))
df_output = fetch_output_2024_prices_mn_eur(config=out_cfg)

# --- 3) Build intensity panel + percentile table -----------------------------
intensity_panel = build_intensity_panel(
    df_type=df_type,
    df_year=df_year,
    df_output=df_output,
    geo_col_output="geo",
)
pct_table = build_percentile_table(intensity_panel)

display(pct_table)

level,ISO2,hazard,extreme,moderate,severe,very_extreme
0,AL,climatological_heatwaves,0.000000,0.000000e+00,0.000000,0.000000
1,AL,climatological_other,0.000147,1.590982e-06,0.000026,0.000245
2,AL,geotechnical,0.002810,3.041910e-05,0.000493,0.004692
3,AL,hydrological,0.000844,9.142712e-06,0.000148,0.001410
4,AL,meteorological,0.000020,2.204903e-07,0.000004,0.000034
...,...,...,...,...,...,...
175,SK,climatological_heatwaves,0.000414,1.640885e-05,0.000069,0.000700
176,SK,climatological_other,0.000000,0.000000e+00,0.000000,0.000000
177,SK,geotechnical,0.000019,7.707190e-07,0.000003,0.000033
178,SK,hydrological,0.001594,6.313950e-05,0.000267,0.002693


In [14]:
# Export to Excel
export_path = "data/checks/eurostat_output_2024_prices.xlsx"

from pathlib import Path
Path(export_path).parent.mkdir(parents=True, exist_ok=True)

df_output.sort_values(["geo", "year"]).to_excel(export_path, index=False)

print(f"Exported df_output to {export_path}")


Exported df_output to data/checks/eurostat_output_2024_prices.xlsx


In [7]:
# --- 4) Choose a hazard scenario -------------------------------------------
haz_country = "IT"                      # ISO2
hazard_type = "meteorological"            # geotechnical / meteorological / hydrological / climatological_other / climatological_heatwaves
intensity_level = "very_extreme"              # moderate / severe / extreme / very_extreme

phi = shock_scalar(
    pct_table,
    country_iso2=haz_country,
    hazard=hazard_type,
    intensity_level=intensity_level,
    clamp_max=0.50,   # safety cap (50% capacity loss)
)
print(f"Selected: {haz_country=} {hazard_type=} {intensity_level=}")
print(f"Supply shock scalar φ = {phi:.4%}")

supply_shock_pct = 100.0 * phi


Selected: haz_country='IT' hazard_type='meteorological' intensity_level='very_extreme'
Supply shock scalar φ = 0.0333%


In [8]:
# --- 5) Run the model (hazard-calibrated supply shock) ----------------------
scenario_hazard = dict(
    supply_country_codes=[haz_country],
    supply_sector_codes=None,       # None => all sectors in that country
    supply_shock_pct=supply_shock_pct,

    demand_country_codes=None,      # keep demand shock off
    demand_sector_codes=None,
    demand_shock_pct=0.0,

    gamma=0.05,
    max_iter=200,
    tol=1e-4,
    return_history=True,
)

results = model.run(**scenario_hazard)

print("Converged:", results["converged"])
print("Iterations:", results["iterations"])


Converged: True
Iterations: 2


In [10]:
# OLD SCENARIO FOR TESTING PURPOSES ONLY

scenario_old = dict(
    # Supply shock target: Portugal energy/utilities
    supply_country_codes=["PT"],      # Portugal
    supply_sector_codes=["P_D35"],        # Electricity, gas, steam
    supply_shock_pct=5.0,

    # Demand shock target: Italy, several service sectors
    demand_country_codes=["IT"],            # Italy
    demand_sector_codes=["P_H49", "P_J61", "P_M69_70"],  # transport, telecom, professional services
    demand_shock_pct=2.5,

    gamma=0.5,
    max_iter=100,
    tol=1e-3,
    return_history=True,
)

results_old = model.run(**scenario_old)

print("Converged:", results_old["converged"])
print("Iterations:", results_old["iterations"])


Converged: True
Iterations: 3


## 4) Build node-level impact table

We compute losses (and gains) in terms of output, value added, and structural change at node level.
The post-processing step produces standardized tables:

- `df_nodes`: node-level impacts (country–sector)
- `df_country`: country aggregates
- `df_sector`: sector aggregates (decoded names if available)
- `df_links_weakened` / `df_links_strengthened`: largest linkage changes (ΔA by default)

Units:
- Output and value added are in the same unit as the SAM flows (typically EUR).


In [11]:
# Postprocess run outputs into standardized tables and KPIs
pp = postprocess_results(
    node_labels=node_labels,
    Z0=Z,
    X0=X_baseline,
    Z1=results["Z_final"],
    X1=results["X_supply_final"],
    FD_post=results.get("FD_post_final"),
    sector_name_map=sector_decoder,   # optional; comment out if not available
    linkage_metric="A",              # structural change via technical coefficients
    top_k_links=25,
)

df_nodes = pp.df_nodes
df_country = pp.df_country
df_sector = pp.df_sector

# Structural change (top linkages)
df_links_weakened = pp.df_links_weakened
df_links_strengthened = pp.df_links_strengthened

display(pp.meta)


{'X_baseline_total': 28772842.248214405,
 'X_final_total': 28763097.788470015,
 'X_loss_abs_total': 9744.4597443901,
 'X_loss_pct_total': 0.033866865359798874,
 'VA_baseline_total': 14831340.77324824,
 'VA_final_total': 14826343.581003373,
 'VA_loss_abs_total': 4997.192244866863,
 'VA_loss_pct_total': 0.03369346252147653,
 'FD_post_total': 14826339.769900825}

In [12]:
# Displaying all tables with their top 10 rows
print("Most Impacted Nodes")
display(df_nodes.head(10))

print("\nMost Impacted Countries")
display(df_country.head(10))

print("\nMost Impacted Sectors")
display(df_sector.head(10))

print("\nTop Weakened Linkages (most negative ΔA)")
display(df_links_weakened.head(10))

print("\nTop Strengthened Linkages (most positive ΔA)")
display(df_links_strengthened.head(10))

Most Impacted Nodes


,node,country,sector,X_baseline,X_final,loss_abs,loss_pct,sector_name,VA_baseline,VA_final,VA_loss_abs,VA_loss_pct
0,AT::P_A01,AT,P_A01,10170.862001,10167.310695,3.551306,0.034916,"Crop and animal production, hunting and relate...",4386.116991,4384.585514,1.531477,0.034916
1,AT::P_A02,AT,P_A02,2903.627000,2902.567330,1.059670,0.036495,Forestry and logging,1247.327498,1246.872289,0.455208,0.036495
2,AT::P_A03,AT,P_A03,111.881000,111.842723,0.038277,0.034212,Fishing and aquaculture,38.568560,38.555365,0.013195,0.034212
3,AT::P_B,AT,P_B,3452.811000,3451.623483,1.187517,0.034393,P_B,1668.297402,1667.723628,0.573774,0.034393
4,AT::P_C10-12,AT,P_C10-12,28074.827001,28065.277245,9.549756,0.034015,Manufacture of food products; beverages and to...,7311.576528,7309.089468,2.487060,0.034015
5,AT::P_C13-15,AT,P_C13-15,2348.636000,2347.839438,0.796562,0.033916,"Manufacture of textiles, wearing apparel, leat...",884.348130,884.048195,0.299935,0.033916
6,AT::P_C16,AT,P_C16,10829.488001,10825.747661,3.740340,0.034538,Manufacture of wood and of products of wood an...,3554.409379,3553.181740,1.227639,0.034538
7,AT::P_C17,AT,P_C17,7605.928001,7603.279299,2.648701,0.034824,Manufacture of paper and paper products,2822.957457,2821.974385,0.983071,0.034824
8,AT::P_C18,AT,P_C18,1935.736000,1935.062586,0.673414,0.034789,Printing and reproduction of recorded media,796.066210,795.789270,0.276940,0.034789
9,AT::P_C19,AT,P_C19,6834.943001,6832.565147,2.377854,0.034790,Manufacture of coke and refined petroleum prod...,1996.614514,1995.919898,0.694615,0.034790



Most Impacted Countries


,country,X_baseline,X_final,loss_abs,VA_baseline,VA_final,VA_loss_abs,loss_pct,VA_loss_pct
5,DE,6.905942e+06,6.903595e+06,2347.010549,3.557843e+06,3.556637e+06,1205.164193,0.033985,0.033873
11,FR,4.681131e+06,4.679534e+06,1596.777895,2.380209e+06,2.379403e+06,806.515578,0.034111,0.033884
15,IT,3.751974e+06,3.750732e+06,1242.608927,1.827599e+06,1.826994e+06,604.200026,0.033119,0.033060
9,ES,2.442810e+06,2.441986e+06,823.964230,1.324897e+06,1.324453e+06,444.049027,0.033730,0.033516
20,NL,1.738425e+06,1.737836e+06,588.908873,9.607036e+05,9.603795e+05,324.147450,0.033876,0.033741
21,PL,1.310212e+06,1.309765e+06,447.025081,6.443907e+05,6.441716e+05,219.077154,0.034119,0.033998
1,BE,1.099704e+06,1.099330e+06,373.694646,5.446183e+05,5.444341e+05,184.181437,0.033981,0.033818
24,SE,9.215626e+05,9.212499e+05,312.672593,4.986103e+05,4.984417e+05,168.576620,0.033929,0.033809
0,AT,8.110387e+05,8.107618e+05,276.833554,4.027755e+05,4.026392e+05,136.327760,0.034133,0.033847
14,IE,6.134451e+05,6.132390e+05,206.176881,4.235278e+05,4.233856e+05,142.143024,0.033610,0.033562



Most Impacted Sectors


,sector,sector_name,X_baseline,X_final,loss_abs,VA_baseline,VA_final,VA_loss_abs,loss_pct,VA_loss_pct
26,P_F,Construction,2.166042e+06,2.165312e+06,730.203613,8.305130e+05,8.302331e+05,279.948156,0.033711,0.033708
43,P_L,Real estate activities,1.951634e+06,1.950976e+06,657.994988,1.502713e+06,1.502207e+06,506.360438,0.033715,0.033696
28,P_G46,"Wholesale trade, except of motor vehicles and ...",1.541742e+06,1.541221e+06,520.917023,8.592338e+05,8.589434e+05,290.423161,0.033788,0.033800
53,P_O84,Public administration and defence; compulsory ...,1.367827e+06,1.367376e+06,450.433010,1.005180e+06,1.004849e+06,331.234198,0.032931,0.032953
23,P_D35,"Electricity, gas, steam and air conditioning s...",1.140940e+06,1.140539e+06,401.333598,4.388395e+05,4.386860e+05,153.477480,0.035176,0.034973
55,P_Q86,Human health activities,1.168269e+06,1.167884e+06,385.105196,8.576227e+05,8.573397e+05,282.952985,0.032964,0.032993
4,P_C10-12,Manufacture of food products; beverages and to...,1.127953e+06,1.127569e+06,384.471636,2.670698e+05,2.669785e+05,91.307052,0.034086,0.034188
29,P_G47,"Retail trade, except of motor vehicles and mot...",1.047942e+06,1.047593e+06,348.754260,6.169385e+05,6.167330e+05,205.484389,0.033280,0.033307
44,P_M69_70,Legal and accounting activities; activities of...,9.442642e+05,9.439344e+05,329.765212,5.080800e+05,5.079030e+05,177.016184,0.034923,0.034840
39,P_J62_63,"Computer programming, consultancy, and informa...",8.213425e+05,8.210618e+05,280.702030,4.561236e+05,4.559677e+05,155.905638,0.034176,0.034181



Top Weakened Linkages (most negative ΔA)


,i_node,j_node,i_label,j_label,baseline,final,delta,delta_rel
0,967,967,IT::P_D35,IT::P_D35,0.507164,0.507163,-4.467919e-07,-8.809618e-07
1,946,1194,IT::P_A03,MT::P_A03,0.098583,0.098583,-1.756649e-07,-1.781901e-06
2,953,953,IT::P_C19,IT::P_C19,0.263499,0.263499,-1.451577e-07,-5.508841e-07
3,973,1140,IT::P_G47,LV::P_C19,0.017824,0.017824,-1.264535e-07,-7.094684e-06
4,982,982,IT::P_J61,IT::P_J61,0.215962,0.215962,-1.239665e-07,-5.740199e-07
5,948,1193,IT::P_C10-12,MT::P_A01,0.022152,0.022152,-1.209219e-07,-5.458709e-06
6,948,948,IT::P_C10-12,IT::P_C10-12,0.218586,0.218586,-1.165560e-07,-5.332266e-07
7,970,970,IT::P_F,IT::P_F,0.263876,0.263876,-1.151511e-07,-4.363826e-07
8,948,979,IT::P_C10-12,IT::P_I,0.221916,0.221916,-1.148986e-07,-5.177572e-07
9,974,978,IT::P_H49,IT::P_H53,0.182879,0.182879,-1.041590e-07,-5.695509e-07



Top Strengthened Linkages (most positive ΔA)


,i_node,j_node,i_label,j_label,baseline,final,delta,delta_rel
0,215,220,CY::P_G45,CY::P_H51,0.334383,0.334385,1.665103e-06,4.979624e-06
1,237,220,CY::P_N77,CY::P_H51,0.137559,0.137559,6.849897e-07,4.979621e-06
2,223,220,CY::P_I,CY::P_H51,0.132033,0.132033,6.574720e-07,4.979621e-06
3,211,212,CY::P_D35,CY::P_E36,0.414493,0.414494,5.788610e-07,1.396551e-06
4,1111,1129,LU::P_L,LU::P_S96,0.145622,0.145622,5.774212e-07,3.965209e-06
5,1220,1203,MT::P_G46,MT::P_C21,0.214042,0.214043,5.696830e-07,2.661545e-06
6,601,617,ES::P_I,ES::P_N79,0.528463,0.528464,5.090887e-07,9.633383e-07
7,6,950,AT::P_C16,IT::P_C16,0.048536,0.048536,4.727447e-07,9.740139e-06
8,526,527,EL::P_D35,EL::P_E36,0.204309,0.204309,3.735213e-07,1.828219e-06
9,1196,1193,MT::P_C10-12,MT::P_A01,0.158112,0.158113,3.702245e-07,2.341531e-06


## 5) Visualizations

This section provides:
- choropleth map (Europe) of absolute or percent losses by country
- bar charts of absolute losses by country and sector
- bar charts of most strenghtened and weakened trade linkages

If `matplotlib` is not installed in your environment, the bar charts will fall back to tables.


In [13]:
# Build and show dashboard visualizations
bundle = build_dashboard_bundle(
    pp,
    country_metric_for_map="loss_pct",
    top_k_countries=20,
    top_k_sectors=20,
    top_k_links=20,
    use_country_names=True,
)

bundle.figures["country_map"].show()
bundle.figures["top_countries"].show()
bundle.figures["top_sectors"].show()

# Optional linkage charts if you computed them
if "links_strengthened" in bundle.figures:
    bundle.figures["links_strengthened"].show()
    bundle.figures["links_weakened"].show()


## 6) Diagnostics: demand adjustment loop

The model iterates on post-shock final demand `FD_post` until it matches the feasible implied demand.
Use this section to verify:
- total demand contraction
- how many outer iterations were needed


In [ ]:
FD_post_final = results["FD_post_final"]
FD_implied_final = results["FD_implied_final"]

print("Total baseline FD:", float(FD_baseline.sum()))
print("Total final FD_post:", float(FD_post_final.sum()))
print("Total final implied FD:", float(FD_implied_final.sum()))
print("Max unmet FD (nodewise):", float(np.max(FD_post_final - FD_implied_final)))
print("Max slack FD (nodewise):", float(np.max(FD_implied_final - FD_post_final)))

# Optional: inspect the demand history (outer loop)
if "FD_post_history" in results:
    totals = [float(v.sum()) for v in results["FD_post_history"]]
    print("FD_post totals by outer iteration:", totals)
